In [2]:
import numpy as np

# Funkcja aktywacji (progowa)
def threshold_function(x: np.array, threshold: int = 0) -> np.array:
    """
    Funkcja progowa (threshold function).

    Dla wartości wejściowej x:
    - zwraca 1, jeśli x >= threshold,
    - zwraca 0, jeśli x < threshold.

    Parametry
    ---------
    x : np.array lub float
        Wartość (lub tablica wartości) do sprawdzenia.
    threshold : int, domyślnie 0
        Próg aktywacji.

    Zwraca
    -------
    np.array
        Wartości 0 lub 1 po zastosowaniu funkcji progowej.
    """
    return np.where(x >= threshold, 1, 0)


class Perceptron:
    """
    Implementacja prostego perceptronu binarnego (uczenie nadzorowane).

    Perceptron uczy się liniowej granicy decyzyjnej dla klasyfikacji
    na podstawie wektorów cech i etykiet binarnych (0 lub 1).
    """

    def __init__(self, learning_rate=0.01, epochs=100,
                 activation_function=threshold_function,
                 start_weights=None, start_bias=None):
        """
        Inicjalizacja perceptronu.

        Parametry
        ---------
        learning_rate : float
            Współczynnik uczenia (krok aktualizacji wag).
        epochs : int
            Liczba epok, tj. ile razy przechodzimy przez cały zbiór treningowy.
        activation_function : callable
            Funkcja aktywacji (np. progowa, sigmoid).
        start_weights : np.array lub None
            Wagi początkowe (jeśli None, będą zainicjalizowane zerami).
        start_bias : float lub None
            Bias początkowy (jeśli None, startuje od 0).
        """
        # hiperparametry
        self.lr = learning_rate
        self.epochs = epochs
        self.activation_func = activation_function

        # parametry (uczone podczas treningu)
        self.weights = start_weights
        self.bias = start_bias or 0

    def fit(self, features, labels):
        """
        Trenuje perceptron na podanych danych.

        Parametry
        ---------
        features : np.array, shape (n_samples, n_features)
            Macierz cech (wejścia).
        labels : np.array lub lista, shape (n_samples,)
            Etykiety klas (0 lub 1).
        """
        n_samples, n_features = features.shape

        # Inicjalizacja wag zerami (jeśli nie podano startowych)
        self.weights = np.zeros(n_features)

        y = np.array(labels)

        # Pętla ucząca (epoki)
        for _ in range(self.epochs):
            # Iterujemy po wszystkich próbkach
            for i, x_i in enumerate(features):
                # Obliczamy wyjście liniowe
                linear_output = self._linear_output(x_i)

                # Przepuszczamy przez funkcję aktywacji
                y_predicted = self.activation_func(linear_output)

                # Obliczamy błąd (y_true - y_pred)
                # i wyznaczamy wielkość aktualizacji
                update = self.lr * (y[i] - y_predicted)

                # Aktualizacja wag i biasu
                self.weights += update * x_i
                self.bias += update

    def _linear_output(self, features):
        """
        Oblicza wyjście liniowe (iloczyn skalarny wejścia i wag + bias).

        Parametry
        ---------
        features : np.array, shape (n_features,)
            Wektor cech dla jednej próbki.

        Zwraca
        -------
        float
            Wartość liniowej kombinacji cech.
        """
        return features @ self.weights + self.bias
    
    def predict(self, features):
        """
        Przewiduje etykiety dla nowych danych.

        Parametry
        ---------
        features : np.array, shape (n_samples, n_features) lub (n_features,)
            Dane wejściowe.

        Zwraca
        -------
        np.array
            Predykcje (0 lub 1) dla próbek wejściowych.
        """
        linear_output = self._linear_output(features)
        y_predicted = self.activation_func(linear_output)
        return y_predicted


W danych masz tylko wzrost i wagę. Musisz je wczytac i przytgotowac sobie labelki
Funkcja wczytujaca powinna zwrocić cechy i lableki

```
def load_data(filepath):
    ...

    return features, labels

In [22]:
# napisz funkcję, która wczyta dane  i przygotowuje jedo treningu perceptronu.

def load_data(filepath: str):
    """
    Wczytuje dane z pliku tekstowego wzrost_waga.txt i przygotowuje je
    do treningu perceptronu.

    Plik powinien mieć format CSV z kolumnami:
        wzrost, waga, etykieta
    gdzie etykieta = 0 lub 1.

    Parametry
    ---------
    filepath : str
        Ścieżka do pliku z danymi.

    Zwraca
    -------
    features : np.array, shape (n_samples, 2)
        Macierz cech [wzrost, waga].
    labels : np.array, shape (n_samples,)
        Etykiety klas (0 lub 1).
    """
    data = np.loadtxt(filepath, delimiter=",", skiprows=1)  # pomijamy nagłówek
    heights = data[:, 0] / 100 # bo to ma być w metrach
    weights = data[:, 1]

    # BMI = waga / (wzrost^2)
    bmi = weights / (heights ** 2)

    # klasy na podstawie progu
    labels = np.where(bmi >= 25, 1, 0)

    features = data  # wzrost, waga
    return features, labels


def load_data(filepath: str):
    """
    Wczytuje dane z pliku CSV (np. wzrost_waga.txt) i przygotowuje je
    do treningu perceptronu.

    Plik powinien mieć format CSV z kolumnami:
        wzrost, waga, etykieta (opcjonalnie)
    gdzie etykieta = 0 lub 1.

    Parametry
    ---------
    filepath : str
        Ścieżka do pliku z danymi.

    Zwraca
    -------
    features : np.ndarray, shape (n_samples, 2)
        Macierz cech [wzrost, waga].
    labels : np.ndarray, shape (n_samples,)
        Etykiety klas (0 lub 1).
        Jeśli w pliku nie ma kolumny "etykieta", zostaną wyliczone na podstawie BMI.
    """
    df = pd.read_csv(filepath)

    # konwersja wzrostu na metry
    df["wzrost_m"] = df["wzrost"] / 100  

    # obliczenie BMI
    df["bmi"] = df["waga"] / (df["wzrost_m"] ** 2)

    # jeśli kolumna "etykieta" istnieje, używamy jej,
    # jeśli nie – tworzymy na podstawie progu BMI
    if "etykieta" in df.columns:
        labels = df["etykieta"].to_numpy()
    else:
        labels = np.where(df["bmi"] >= 25, 1, 0)

    # cechy: tylko wzrost i waga
    features = df[["wzrost", "waga"]].to_numpy()

    return features, labels

In [23]:
# 1. Wczytaj dane
X, y = load_data("wzrost_waga.txt")
y

array([0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 0])

In [24]:
X

array([[162.,  48.],
       [163.,  60.],
       [182.,  60.],
       [187., 107.],
       [169.,  59.],
       [165.,  60.],
       [168.,  67.],
       [179., 107.],
       [157.,  75.],
       [160.,  85.],
       [167.,  62.],
       [173.,  72.],
       [179., 100.],
       [165.,  74.],
       [159.,  55.],
       [165.,  57.],
       [175.,  71.],
       [157.,  56.],
       [168.,  87.],
       [168.,  82.],
       [164.,  74.],
       [182.,  77.],
       [163.,  56.],
       [162.,  70.],
       [159.,  78.],
       [188.,  69.],
       [162.,  80.],
       [176.,  91.],
       [155.,  81.],
       [187.,  79.]])

In [16]:
# 2. Utwórz i wytrenuj perceptron
p = Perceptron(learning_rate=0.01, epochs=100)
p.fit(X, y)

In [28]:
# 3. sprawdz na danych testowych jak dziala model
# Dane: [wzrost (m), waga (kg), klasa]
# Klasa obliczona na podstawie BMI (próg = 25)
data = np.array([
    [160, 30, 0],   # BMI = 19.5 → klasa 0
    [175, 68, 0],   # BMI = 22.2 → klasa 0
    [180, 90, 1],   # BMI = 27.8 → klasa 1
    [165, 80, 1],   # BMI = 29.4 → klasa 1
    [190, 95, 1],   # BMI = 26.3 → klasa 1
    [170, 60, 0],   # BMI = 20.8 → klasa 0
    [155, 70, 1],   # BMI = 29.1 → klasa 1
])

print(data[:,1] /data[:,0] ** 2)

# Podział na cechy (X) i etykiety (y)
X_test = data[:, :2]   # wzrost, waga
y_test = data[:, 2]    # klasa

X_test


[0.00117187 0.00222041 0.00277778 0.00293848 0.00263158 0.00207612
 0.00291363]


array([[160,  30],
       [175,  68],
       [180,  90],
       [165,  80],
       [190,  95],
       [170,  60],
       [155,  70]])

In [29]:
predictions = p.predict(X_test)
predictions

array([0, 0, 1, 1, 1, 0, 1])

In [27]:
y_test

array([0., 0., 1., 1., 1., 0., 1.])

In [30]:


def confusion_matrix(y_true, y_pred):
    """
    Oblicza macierz pomyłek (confusion matrix) dla klasyfikacji binarnej.

    Parametry
    ---------
    y_true : np.array
        Rzeczywiste etykiety (0 lub 1).
    y_pred : np.array
        Predykcje modelu (0 lub 1).

    Zwraca
    -------
    dict
        Słownik z wartościami: TP, TN, FP, FN oraz macierz 2x2.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))

    matrix = np.array([[TN, FP],
                       [FN, TP]])

    return {
        "TN": TN, "FP": FP,
        "FN": FN, "TP": TP,
        "matrix": matrix
    }


In [31]:
confusion_matrix(y_test, predictions)

{'TN': 3,
 'FP': 0,
 'FN': 0,
 'TP': 4,
 'matrix': array([[3, 0],
        [0, 4]])}

In [34]:
def accuracy_score(y_true, y_pred):
    """
    Accuracy = (TP + TN) / (TP + TN + FP + FN)
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.mean(y_true == y_pred)


def precision_score(y_true, y_pred):
    """
    Precision = TP / (TP + FP)
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    TP = np.sum((y_true == 1) & (y_pred == 1))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    return TP / (TP + FP) if (TP + FP) > 0 else 0.0


In [36]:
accuracy_score(y_test, predictions)

1.0

In [37]:
precision_score(y_test, predictions)

1.0